In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        new_V = np.copy(V)
        for s in range(env.nS):
            v = 0
            for a in range(env.nA):
                action_prob = policy[s][a]
                immediate_reward = env.P[s][a][0][2]
                v_prim = 0
                for transition_tuple in env.P[s][a]:
                    next_state = transition_tuple[1]
                    transition_prob = transition_tuple[0]
                    v_prim += discount_factor * transition_prob * V[next_state]
                v += action_prob * (immediate_reward + v_prim)
            new_V[s] = v
        if (abs(V - new_V) < theta).all():
            #print(V.reshape(env.shape))
            break
        V = new_V
    return np.array(V)

In [38]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: Lambda discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        # Implement this!
        V = policy_eval_fn(policy, env, discount_factor=1.0, theta=0.00001)
        print('Value function:')
        print(V.reshape(env.shape))
        print('Policy function:')
        print(policy)
        policy_stable = True
        for s in range(env.nS):
            best_action = np.argmax(policy[s])
            q_value = np.zeros(env.nA)
            for a in range(env.nA):
                v_prim = 0
                immediate_reward = env.P[s][a][0][2]
                for transition_tuple in env.P[s][a]:
                    next_state = transition_tuple[1]
                    transition_prob = transition_tuple[0]
                    v_prim += discount_factor * transition_prob * V[next_state]
                q_value[a] = immediate_reward + v_prim
            #print('q_value:')
            #print(q_value)
            #print('s = %d, chosen_action = %s' %(s, chosen_action))
            chosen_action = np.argmax(q_value)
            policy[s] = np.zeros(policy[s].shape)
            policy[s][chosen_action] = 1
            if chosen_action != best_action:
                policy_stable = False
        if policy_stable:
            break    
    return policy, V

In [39]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



Value function:
[[  0.         -13.99988715 -19.99983277 -21.99981286]
 [-13.99988715 -17.99985268 -19.99983389 -19.99983277]
 [-19.99983277 -19.99983389 -17.99985268 -13.99988715]
 [-21.99981286 -19.99983277 -13.99988715   0.        ]]
Policy function:
[[ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]]
Value function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Policy function:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.

In [40]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)